# Importing Required Libraries

In [1]:
import pickle
import numpy as np
import nltk
import json
import random
from nltk.stem import WordNetLemmatizer
from tkinter import *

f:\mentalhealthcanada\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
f:\mentalhealthcanada\venv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
f:\mentalhealthcanada\venv\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Importing the created model

In [2]:
#from tenshttp://localhost:8890/notebooks/mentalhealthbot.ipynb
#Importing-the-created-modelorflow.keras.models import load_model
from tensorflow.keras.models import load_model
createbot = load_model('mentalhealthbot_model.h5')

# Retrieving the intents, words and classes

In [3]:
intents = json.loads(open('mentalhealthCanada.json').read())
vocabulary = pickle.load(open('vocabulary.pkl','rb'))
categories = pickle.load(open('categories.pkl','rb'))

# Text pre-processing (Tokenizing and lemmatizing each word)

In [4]:
def preprocessing(sentence):
    
    # Tokenizing the pattern - splitting words into an array
    sentences = nltk.word_tokenize(sentence)
    
    # Lemmatizing each word - creating short forms (lemma) for words
    lemma = WordNetLemmatizer()
    sentences = [lemma.lemmatize(word.lower()) for word in sentences]
    return sentences

# Returning the bag of words array = '0' or '1' for each word in the bag of words that exists in the sentence

In [5]:
def bagofwords(sentence, words, details=True):
    
    # Tokenizing the pattern
    sentences = preprocessing(sentence)
    
    # Bag of words - matrix of N words which is the vocabulary matrix
    bagofwords = [0]*len(words)  
    for s in sentences:
        for i,w in enumerate(words):
            if w == s: 
                
                # Assigning 1 if the current word exists in the vocabulary position
                bagofwords[i] = 1
                if details:
                    print ("found in bag: %s" % w)
    return(np.array(bagofwords))

# Defining the prediction classes

In [6]:
def prediction(sentence, model):
    
    # Filtering out predictions below a threshold of 0.25
    predict = bagofwords(sentence, vocabulary,details=False)
    res = model.predict(np.array([predict]))[0]
    error_threshold = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error_threshold ]
    
    # Sorting the results by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": categories[r[0]], "probability": str(r[1])})
    return return_list

# Defining the responses

In [7]:
def response(ints, intents_json):
    tag = ints[0]['intent']
    intents_list = intents_json['intents']
    for i in intents_list:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

# Defining chatbot responses based on users input queries

In [8]:
def chatbot_response(msg):
    ints = prediction(msg, createbot)
    res = response(ints, intents)
    return res

# Creating GUI for the Mental Health Chatbot using tkinter

In [9]:
# Chatbot GUI code adapted from Analytics Vidhya
# https://www.analyticsvidhya.com/blog/2021/06/learn-to-develop-a-simple-chatbot-using-python-and-deep-learning/

def send():
    message = input_box.get("1.0",'end-1c').strip()
    input_box.delete("0.0",END)

    if message != '':
        chat.config(state=NORMAL)
        chat.insert(END, "You: " + message + '\n\n')
        chat.config(foreground="#442265", font=("Verdana", 12), wrap='word')
    
        res = chatbot_response(message)
        chat.insert(END, "MHAC: " + res + '\n\n')
            
        chat.config(state=DISABLED)
        chat.yview(END)

base = Tk()
base.title("Mental Health Awareness Canada")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

''

# Creating the Display Window

In [10]:
chat = Text(base, bd=0, bg="#FBFCFC", height="8", width="50", font="Verdana",)

chat.config(state=DISABLED)

# Binding a Scrollbar to the Display Window

In [11]:
scrollbar = Scrollbar(base, command=chat.yview, cursor="heart")
chat['yscrollcommand'] = scrollbar.set

# Creating a 'Send' Button for users to send messages

In [12]:
send_button = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#F06292", activebackground="#F06292",fg='#ffffff',
                    command= send )

# Creating the input box for users to enter messages

In [13]:
input_box = Text(base, bd=0, bg="white",width="29", height="5", font="Arial", wrap='word')
input_box.bind("<Return>", send)

'2385869228224send'

# Placing all the components for the Chatbot GUI

In [14]:
input_box.place(x=6, y=440, height=50, width=250)
scrollbar.place(x=376,y=6, height=386)
chat.place(x=6,y=6, height=425, width=370)
send_button.place(x=245, y=440, height=50)
base.mainloop()